In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requi

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
import os
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Toxicity_data_Mistral_v3_forFT.csv to Toxicity_data_Mistral_v3_forFT.csv


In [ ]:
dat0 = pd.read_csv('Toxicity_data_Mistral_v3_forFT.csv')

In [ ]:
dat0.shape

(2500, 2)

In [ ]:
train_dat = dat0.sample(frac=0.6, random_state=42)

In [ ]:
train_dat.shape

(1500, 2)

In [ ]:
train_dat['label'].value_counts()

label
2    745
1    455
0    300
Name: count, dtype: int64

In [ ]:
rest_part_50 = dat0.drop(train_dat.index)

In [ ]:
rest_part_50.shape

(1000, 2)

In [ ]:
eval_dat = rest_part_50.sample(frac=0.5, random_state=42)

In [ ]:
test_dat = rest_part_50.drop(eval_dat.index)

In [ ]:
eval_dat.shape

(500, 2)

In [ ]:
test_dat.shape

(500, 2)

In [ ]:
eval_dat['label'].value_counts()

label
2    246
1    154
0    100
Name: count, dtype: int64

In [ ]:
test_dat['label'].value_counts()

label
2    259
1    141
0    100
Name: count, dtype: int64

In [ ]:
train_dataset = Dataset.from_pandas(train_dat)
eval_dataset = Dataset.from_pandas(eval_dat)
test_dataset = Dataset.from_pandas(test_dat)

In [ ]:
# Load model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_datasets_test = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_eval,
)

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.010067
2,No log,0.003238
3,0.136500,0.001361


TrainOutput(global_step=564, training_loss=0.1211563044670203, metrics={'train_runtime': 219.575, 'train_samples_per_second': 20.494, 'train_steps_per_second': 2.569, 'total_flos': 1184010379776000.0, 'train_loss': 0.1211563044670203, 'epoch': 3.0})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("transform-satrajit/bert-b-u-toxic-bias-ft1", use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:875: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/transform-satrajit/bert-b-u-toxic-bias-ft1/commit/5ceafb9665c674be6ebb645eff67ff577a7d6d63', commit_message='Upload BertForSequenceClassification', commit_description='', oid='5ceafb9665c674be6ebb645eff67ff577a7d6d63', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenized_datasets_train

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1500
})

In [ ]:
test_dataset = Dataset.from_pandas(test_dat)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove the text column as it won't be used for prediction
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text", "__index_level_0__"])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

# Create DataCollator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create DataLoader for test dataset
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=8, collate_fn=data_collator)

# Put the model in evaluation mode
model.eval()

predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs = {key: val.to(model.device) for key, val in batch.items()}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        predictions.extend(preds)

# Add predictions to test dataframe
test_dat["predictions"] = predictions

# Display the predictions
print(test_dat)


                                                   text  label  predictions
1     2. Listen here, you imbecile! I'm not your per...      1            1
5     1. You'll get your damn credit card when we fe...      1            1
12    3. "Seriously? Another customer losing their c...      1            1
27    3. You're lucky I even remember you have an ac...      1            1
28    4. I can't believe you're still wasting my tim...      1            1
...                                                 ...    ...          ...
2461  2. We anticipate that your credit card will ar...      0            0
2478  9. I'm sorry for any inconvenience this may ca...      0            0
2479  10. Your security is important to us, please r...      0            0
2490  1. Of course, please ensure you have a secure ...      0            0
2496  7. Your online transaction is a simple process...      0            0

[500 rows x 3 columns]


In [ ]:
pd.crosstab(test_dat['label'], test_dat['predictions'])

predictions,0,1,2
label,,,
0,100,0,0
1,0,140,1
2,1,0,258
